# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!Powershell.exe -Command type publications.tsv -Head 3

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-12-23	Changes in plant architecture in Brazilian peppertree damaged by the biological control agent, Pseudophilothrips ichini Hood (Thysanoptera: Phlaeothripidae).		"Halbritter, D. A., E. Kariuki, G. S. Wheeler, M. B. Rayamajhi, C. Minteer, and Q. D. Read. Changes in plant architecture in Brazilian peppertree damaged by the biological control agent, Pseudophilothrips ichini Hood (Thysanoptera: Phlaeothripidae). Biological Control 188, 105414. DOI: 10.1016/j.biocontrol.2023.105434."	halbritter-et-al-2023	https://doi.org/10.1016/j.biocontrol.2023.105434
2023-11-11	Stem borer herbivory dependent on interactions of sugarcane variety, associated traits, and presence of prior borer damage.	Pest Management Science		"Penn, H. J. and Q. D. Read. 2023. Stem borer herbivory dependent on interactions of sugarcane variety, associated traits, and presence of prior borer damage. Pest Management Science. DOI: 10.1002/ps.7843."	penn-and-read

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications.head()


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-12-23,Changes in plant architecture in Brazilian pep...,Biological Control,NaN,"Halbritter, D. A., E. Kariuki, G. S. Wheeler, ...",halbritter-et-al-2023,https://doi.org/10.1016/j.biocontrol.2023.105434
1,2023-11-11,Stem borer herbivory dependent on interactions...,Pest Management Science,NaN,"Penn, H. J. and Q. D. Read. 2023. Stem borer h...",penn-and-read-2023,https://doi.org/10.1002/ps.7843
2,2023-11-02,Distribution of antibiotic resistance in a mix...,Antibiotics,Open access!,"Cho, S., L. M. Hiott, Q. D. Read, J. Damashek,...",cho-et-al-2023,https://doi.org/10.3390/antibiotics12111586
3,2023-10-20,Potential energy and environmental footprint s...,Environmental Science & Technology,Open access!,"Osei-Owusu, K. A., Q. D. Read, and M. Thomsen....",osei-owusu-et-al-2023,https://doi.org/10.1021/acs.est.3c00158
4,2023-08-01,Rye planting date impacts biomass production m...,Agronomy Journal,NaN,"Balkcom, K. S., Q. D. Read, and A. V. Gamble. ...",balkcom-et-al-2023,https://doi.org/10.1002/agj2.21418


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [10]:
# Here, subset the rows of publications to only 1 or however many we need to update right now
publications = publications.head(1)

In [11]:
# Some test code to see what the filename would be
str(publications['pub_date'])

'0    2023-12-23\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [13]:
%%cmd
dir /s /o:-d ..\_publications\ 

Microsoft Windows [Version 10.0.19045.3803]
(c) Microsoft Corporation. All rights reserved.

C:\Users\qdread\Documents\GitHub\qdread.github.io\markdown_generator>dir /s /o:-d ..\_publications\ 
 Volume in drive C has no label.
 Volume Serial Number is 08D0-A2BB

 Directory of C:\Users\qdread\Documents\GitHub\qdread.github.io\_publications

12/27/2023  10:13 AM               777 2023-12-23-halbritter-et-al-2023.md
12/27/2023  10:13 AM    <DIR>          ..
12/27/2023  10:13 AM    <DIR>          .
11/11/2023  03:14 PM               596 2023-11-11-penn-and-read-2023.md
11/02/2023  08:20 AM               882 2023-11-02-cho-et-al-2023.md
10/25/2023  11:57 AM               762 2023-10-20-osei-owusu-et-al-2023.md
10/25/2023  07:36 AM             1,137 2020-06-01-record-et-al-2020-remote-sensing-of-biodiversity.md
08/01/2023  07:39 AM               546 2023-08-01-balkcom-et-al-2023.md
07/22/2023  01:58 PM               791 2023-07-21-degracia-et-al-2023.md
06/14/2023  08:56 PM               600